In [1]:
import socket
import json
import re

import pandas as pd

In [2]:
# Change this parameter for the number of messages you want in the dataset
# I chose 10 just for tests
DATASET_LENGTH = 10

In [3]:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 8080))

In [4]:
channel_list = []

while len(channel_list) < DATASET_LENGTH:
    data = client_socket.recv(1024)
    try: 
        channel_list.append(json.loads(data))
    except:
        data_str = data.decode('utf-8')
        # match curly braces content (payload of json)
        matched = re.findall(r'{(.*?)}', data_str)
        # add braces to convert to json
        # with better regex matching this can be avoided
        matched = list(map(lambda x: '{'+x+'}', matched))
        # re-encode into utf-8 to use json.loads()
        matched = list(map(lambda x: x.encode('utf-8'), matched))
        # convert to dict
        matched = list(map(lambda x: json.loads(x), matched))
        channel_list.extend(matched)
    print(data)

    
client_socket.shutdown(socket.SHUT_RDWR)

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [ ]:
# print(channel_list)
df = pd.DataFrame(channel_list)

df.to_csv('dataset.csv')